In [1]:
import numpy as np 
import pandas as pd
import itertools as it

For a given tree T, we can calculate a distance between two leafs $i$ and $j$, noted as $d_{ij}(T)$

In [2]:
# distance matrix 

m1 = np.array([[0,8,7,12],
               [8,0,9,14],
               [7,9,0,11],
               [12,14,11,0]])

m2 = np.array([[0,2,3,8,14,18],
               [2,0,3,8,14,18],
               [3,3,0,8,14,18],
               [8,8,8,0,14,18],
               [14,14,14,14,0,18],
               [18,18,18,18,18,0]])

In [3]:
def convert_matrix_df(matrix,columns_name=None):
    return pd.DataFrame(matrix, columns=columns_name)

In [4]:
m1_df = convert_matrix_df(m1)
m1_df

,0,1,2,3
0,0,8,7,12
1,8,0,9,14
2,7,9,0,11
3,12,14,11,0


In [5]:
m2_df = convert_matrix_df(m2)
m2_df

,0,1,2,3,4,5
0,0,2,3,8,14,18
1,2,0,3,8,14,18
2,3,3,0,8,14,18
3,8,8,8,0,14,18
4,14,14,14,14,0,18
5,18,18,18,18,18,0


In the context of molecular phylogenetics, an additive matrix is a technique for displaying the evolutionary distances between sequences. This matrix shows the evolutionary changes that have taken place between various biological sequences, including sequences of DNA, RNA, and proteins.
Based on:
* Buneman’s 4-point condition Theorem:  M is additive if and only if the 4-point condition is satisfied
* 3-point condition Theorem: M is ultrametric if and only if the 3-point condition is satisfied

In [6]:
def is_additive(matrix):    
    comb = it.combinations(range(len(matrix)),4)
    for groupe in comb:
        i,j,k,l=groupe
        if not (matrix[i,j]+matrix[k,l]<=max(matrix[i,k]+matrix[j,l],matrix[i,l]+matrix[j,k])):
            return False
    return True

def is_ultrametrix(matrix):
    comb=it.combinations(range(len(matrix)),3)
    for groupe in comb:
        i,j,k=groupe
        if not(matrix[i,k] <= max(matrix[i,j], matrix[j,k])):
            return False
    return True

In [7]:
print("is M1 additive", is_additive(m1))
print("is M2 ultrametrix", is_ultrametrix(m2))

is M1 additive True
is M2 ultrametrix True


In [8]:
def cluster(df_matrix,i):
    return sum(df_matrix.iloc[:, i])

def all_cluster(df_matrix):
    for column in df_matrix.columns:
        print("Number of cluster in ",column,"is",cluster(df_matrix,column)) 

In [9]:
all_cluster(m1_df)

Number of cluster in  0 is 27
Number of cluster in  1 is 31
Number of cluster in  2 is 27
Number of cluster in  3 is 37


In [10]:
all_cluster(m2_df)

Number of cluster in  0 is 45
Number of cluster in  1 is 45
Number of cluster in  2 is 46
Number of cluster in  3 is 56
Number of cluster in  4 is 74
Number of cluster in  5 is 90


---

The Newick format is a technique for representing hierarchical tree structures. It is frequently used in computer science to depict hierarchical connections and in biology to describe phylogenetic trees, which show the evolutionary links between species.

UPGMA steps: 
1. Align & name 
2. Compare sequences using pairwise sequence alignment 
3. Count the mismatches and records them in the mismatche matrix 
4. Create a new cluster $u$ that joins the Closest Pair $(i,j)$ with the smallest distance $d_{i,j}$
5. Update the MatriX replace the rows and columns that correspond to the two clustered items with a new row and column. Based on the average distance from the newly.
6. Repeat step 4 and 5 until we get one cluster  


In [11]:
# Sequences 
a = "ATCGATCG"
b = "GTAGACGA"
c = "ACCGTACG"
d = "TCAGTCAG"
e = "GCCTACAG"

In [12]:
# Compute pairwise sequence alignment
# Both sequences must possess identical lengths.
def pairwise_sequence_alignment(seq_1,seq_2):
    count_missmatch = 0
    for i in range(len(seq_1)):
        if seq_1[i] != seq_2[i]:
            count_missmatch += 1
    return count_missmatch

In [13]:
# Count the mismatches and records them in the mismatche matrix
# min of mismatche_matrix is added in this function since we will use it later 
# outuput: dataframe 
def  mismatche_matrix(seq_list, seq_names):
    n = len(seq_list)
    mismatch_m = np.zeros((n,n))

    for i in range(n):
        sub_seq_list = seq_list[i+1:]
        for j in range(len(sub_seq_list)):
            matrix_j = j+i+1
            pairwise_seq = pairwise_sequence_alignment(seq_list[i],sub_seq_list[j])
            mismatch_m[i][matrix_j] = mismatch_m[matrix_j][i] = pairwise_seq

    mismatch_matrix_df = pd.DataFrame(data=mismatch_m, columns=seq_names ,index=seq_names)
    
    return mismatch_matrix_df

In [14]:
# This function return Minimum value greater than 0 in a dataframe
def min_val_index_df(df): 
    # Finding the minimum value greater than 0
    min_val = df[df > 0].min().min()

    # Getting the indices of the minimum value
    indices = df[df == min_val].stack().index.tolist()
    
    return min_val, list(indices[-1])

In [15]:
# This function calculate the distance between the new cluster and the others
def cal_cluster_distance(df,min_indices):
    cluster_missmatch_socre = []
    cluster_i = min_indices[0]
    cluster_j = min_indices[1]
    for col in df.columns: 
        if col is not cluster_i and col is not cluster_j :
            cluster_missmatch_socre.append((df[col][cluster_i]+df[col][cluster_j]) / 2)
    return cluster_missmatch_socre

In [16]:
def upgma_one_iter(df,dict_tree_newick_format):

    # Find the pairs (i,j)
    min_val, min_indices = min_val_index_df(df)

    # Create new cluster  for pairs (i,j)
    ## intialize the new cluster name u 
    new_cluster_name = min_indices[0]+min_indices[1]
    ## calculate the distance between u_i and u_j
    cluster_branch_distance = min_val/2
    ## update the tree_newick_format 
    i = str(min_indices[0])
    j = min_indices[1]

    if len(min_indices[0]) > 1:
        i = dict_tree_newick_format[min_indices[0]] 
    if len(min_indices[1]) > 1:
        j = dict_tree_newick_format[min_indices[1]] 

    tree_newick_format = "("+i+":"+str(cluster_branch_distance)+","+j+":"+str(cluster_branch_distance)+")"

    if len(new_cluster_name) > 1:
        dict_tree_newick_format[new_cluster_name] = tree_newick_format
        
    ## Compute the distance between the new cluster and the others 
    cluster_missmatch_socre = cal_cluster_distance(df,min_indices)


    # Upadte the mismatch matrix
    ## Delete pairs i j from the df 
    df.drop(min_indices, axis=1, inplace=True)
    df.drop(min_indices, axis=0, inplace=True)

    ## create new col with new cluster name and a new row 
    ### add col
    df[new_cluster_name] = cluster_missmatch_socre
    ### add row 
    cluster_missmatch_socre.append(0)
    df.loc[new_cluster_name] = cluster_missmatch_socre
    
    # return the new mismatch_matrix and the tree_newick_format
    return df,tree_newick_format

----

In [34]:
seq_list = [a,b,c,d,e]
seq_names = ['A','B','C','D','E']
mismatch_matrix_df = mismatche_matrix(seq_list,seq_names)

dict_tree_newick_format = {} 
while len(mismatch_matrix_df) > 1: 
    print(mismatch_matrix_df)
    mismatch_matrix_df,tree_newick_format =  upgma_one_iter(mismatch_matrix_df,dict_tree_newick_format)
    print(tree_newick_format)
    mismatch_matrix_df
    

     A    B    C    D    E
A  0.0  5.0  3.0  6.0  5.0
B  5.0  0.0  7.0  5.0  5.0
C  3.0  7.0  0.0  4.0  5.0
D  6.0  5.0  4.0  0.0  4.0
E  5.0  5.0  5.0  4.0  0.0
(C:1.5,A:1.5)
      B    D    E   CA
B   0.0  5.0  5.0  6.0
D   5.0  0.0  4.0  5.0
E   5.0  4.0  0.0  5.0
CA  6.0  5.0  5.0  0.0
(E:2.0,D:2.0)
      B   CA   ED
B   0.0  6.0  5.0
CA  6.0  0.0  5.0
ED  5.0  5.0  0.0
((E:2.0,D:2.0):2.5,(C:1.5,A:1.5):2.5)
        B  EDCA
B     0.0   5.5
EDCA  5.5   0.0
(((E:2.0,D:2.0):2.5,(C:1.5,A:1.5):2.5):2.75,B:2.75)


----